In [1]:
cd ..

/Users/arthur/Documents/Flatiron/phase_4/Project/pneumonia_CT_scan


In [2]:
import os, shutil, random
import pandas as pd
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
train_dir = 'data/split/train'
val_dir = 'data/split/val'
test_dir = 'data/split/test'

train_pneu = train_dir + '/PNEUMONIA'
val_pneu = val_dir + '/PNEUMONIA'
test_pneu = test_dir + '/PNEUMONIA'
train_non_pneu = train_dir + '/NORMAL'
val_non_pneu = val_dir + '/NORMAL'
test_non_pneu = test_dir + '/NORMAL'

In [4]:
print('There are', len(os.listdir(train_pneu)), 'pneumonia images in the training set')
print('There are', len(os.listdir(val_pneu)), 'pneumonia images in the validation set')
print('There are', len(os.listdir(test_pneu)), 'pneumonia images in the testing set')
print('There are', len(os.listdir(train_non_pneu)), 'non-pneumonia images in the training set')
print('There are', len(os.listdir(val_non_pneu)), 'non-pneumonia images in the validation set')
print('There are', len(os.listdir(test_non_pneu)), 'non-pneumonia images in the testing set')

There are 2564 pneumonia images in the training set
There are 855 pneumonia images in the validation set
There are 854 pneumonia images in the testing set
There are 950 non-pneumonia images in the training set
There are 317 non-pneumonia images in the validation set
There are 316 non-pneumonia images in the testing set


In [5]:
# Get all the data in the directory data/train (2564+950 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_dir, 
        target_size=(64, 64), batch_size=32)

# Get all the data in the directory data/val (855+317 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_dir, 
        target_size=(64, 64), batch_size=32)

# Get all the data in the directory data/test (854+316 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir, 
        target_size=(64, 64), batch_size=32)

# Create the datasets
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)
test_images, test_labels = next(test_generator)

Found 3514 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
Found 1170 images belonging to 2 classes.


In [6]:
print(np.shape(train_images))
print(np.shape(train_labels))
print(np.shape(val_images))
print(np.shape(val_labels))
print(np.shape(test_images))
print(np.shape(test_labels))

(32, 64, 64, 3)
(32, 2)
(32, 64, 64, 3)
(32, 2)
(32, 64, 64, 3)
(32, 2)


In [7]:
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)

{'NORMAL': 0, 'PNEUMONIA': 1}
{'NORMAL': 0, 'PNEUMONIA': 1}
{'NORMAL': 0, 'PNEUMONIA': 1}


In [8]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)
print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(32, 12288)
(32, 12288)
(32, 12288)


In [9]:
train_y = np.reshape(train_labels[:,0], (train_images.shape[0],1))
test_y = np.reshape(test_labels[:,0], (test_images.shape[0],1))
val_y = np.reshape(val_labels[:,0], (val_images.shape[0],1))

## Dummy Model

In [10]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(train_images, train_labels)

DummyClassifier(strategy='most_frequent')

In [11]:
dummy_clf.score(train_images,train_labels)

0.6875

In [12]:
dummy_clf.score(val_images,val_labels)

0.65625

In [13]:
dummy_clf.score(test_images,test_labels)

0.71875

## Model 1: Neural Network without Hidden Layers

In [14]:
np.random.seed(123)
model1 = models.Sequential()
model1.add(layers.Dense(2, activation='relu', input_shape=(12288,)))

In [15]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history1 = model1.fit(train_img,
                     train_y,
                     epochs=50,
                     batch_size=32,
                     validation_data=(val_img, val_y))

Epoch 1/50
1/1 [==============================] - 1s 959ms/step - loss: 2.9876 - accuracy: 0.6562 - val_loss: 7.6548 - val_accuracy: 0.6562
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 7.6521 - accuracy: 0.6875 - val_loss: 7.6548 - val_accuracy: 0.6562
Epoch 3/50
1/1 [==============================] - 0s 33ms/step - loss: 7.6521 - accuracy: 0.6875 - val_loss: 7.6548 - val_accuracy: 0.6562
Epoch 4/50
1/1 [==============================] - 0s 34ms/step - loss: 7.6521 - accuracy: 0.6875 - val_loss: 7.6548 - val_accuracy: 0.6562
Epoch 5/50
1/1 [==============================] - 0s 34ms/step - loss: 7.6521 - accuracy: 0.6875 - val_loss: 7.6548 - val_accuracy: 0.6562
Epoch 6/50
1/1 [==============================] - 0s 35ms/step - loss: 7.6521 - accuracy: 0.6875 - val_loss: 7.6548 - val_accuracy: 0.6562
Epoch 7/50
1/1 [==============================] - 0s 35ms/step - loss: 7.6521 - accuracy: 0.6875 - val_loss: 7.6548 - val_accuracy: 0.6562
Epoch 8/50
1/1 [==========

In [16]:
results_train = model1.evaluate(train_img, train_y)

1/1 [==============================] - 0s 16ms/step - loss: 7.6521 - accuracy: 0.6875


In [17]:
results_test = model1.evaluate(test_img, test_y)

1/1 [==============================] - 0s 15ms/step - loss: 7.6493 - accuracy: 0.7188


In [18]:
results_train

[7.652073860168457, 0.6875]

In [19]:
results_test

[7.649328231811523, 0.71875]

## Model 2: Neural Network with Dense Layers

In [20]:
np.random.seed(123)
model2 = models.Sequential()
model2.add(layers.Dense(200, activation='relu', input_shape=(12288,)))
model2.add(layers.Dense(100, activation='relu'))
model2.add(layers.Dense(50, activation='relu'))
model2.add(layers.Dense(25, activation='relu'))
model2.add(layers.Dense(7, activation='relu'))
model2.add(layers.Dense(5, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

In [21]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history2 = model2.fit(train_img,
                     train_y,
                     epochs=50,
                     batch_size=32,
                     validation_data=(val_img, val_y))

Epoch 1/50
1/1 [==============================] - 1s 518ms/step - loss: 0.6927 - accuracy: 0.6875 - val_loss: 0.6925 - val_accuracy: 0.6562
Epoch 2/50
1/1 [==============================] - 0s 40ms/step - loss: 0.6923 - accuracy: 0.6875 - val_loss: 0.6921 - val_accuracy: 0.6562
Epoch 3/50
1/1 [==============================] - 0s 41ms/step - loss: 0.6918 - accuracy: 0.6875 - val_loss: 0.6917 - val_accuracy: 0.6562
Epoch 4/50
1/1 [==============================] - 0s 42ms/step - loss: 0.6914 - accuracy: 0.6875 - val_loss: 0.6914 - val_accuracy: 0.6562
Epoch 5/50
1/1 [==============================] - 0s 41ms/step - loss: 0.6909 - accuracy: 0.6875 - val_loss: 0.6910 - val_accuracy: 0.6562
Epoch 6/50
1/1 [==============================] - 0s 41ms/step - loss: 0.6905 - accuracy: 0.6875 - val_loss: 0.6906 - val_accuracy: 0.6562
Epoch 7/50
1/1 [==============================] - 0s 39ms/step - loss: 0.6900 - accuracy: 0.6875 - val_loss: 0.6903 - val_accuracy: 0.6562
Epoch 8/50
1/1 [==========

In [22]:
results_train = model2.evaluate(train_img, train_y)

1/1 [==============================] - 0s 22ms/step - loss: 0.6610 - accuracy: 0.6875


In [23]:
results_test = model2.evaluate(test_img, test_y)

1/1 [==============================] - 0s 17ms/step - loss: 0.6556 - accuracy: 0.7188


In [24]:
results_train

[0.6610368490219116, 0.6875]

In [25]:
results_test

[0.6555837392807007, 0.71875]

## Model 3: Neural Network with Dense and Drop Layers

In [26]:
np.random.seed(123)
model3 = models.Sequential()
model3.add(layers.Dense(200, activation='relu', input_shape=(12288,)))
model3.add(layers.Dense(100, activation='relu'))
model3.add(layers.Dropout(0.3))
model3.add(layers.Dense(50, activation='relu'))
model3.add(layers.Dense(25, activation='relu'))
model3.add(layers.Dropout(0.3))
model3.add(layers.Dense(7, activation='relu'))
model3.add(layers.Dense(5, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))

In [27]:
model3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history3 = model3.fit(train_img,
                     train_y,
                     epochs=50,
                     batch_size=32,
                     validation_data=(val_img, val_y))

Epoch 1/50
1/1 [==============================] - 1s 527ms/step - loss: 0.6790 - accuracy: 0.6875 - val_loss: 0.6862 - val_accuracy: 0.6562
Epoch 2/50
1/1 [==============================] - 0s 37ms/step - loss: 0.6768 - accuracy: 0.6875 - val_loss: 0.6812 - val_accuracy: 0.6562
Epoch 3/50
1/1 [==============================] - 0s 38ms/step - loss: 0.6593 - accuracy: 0.6875 - val_loss: 0.6749 - val_accuracy: 0.6562
Epoch 4/50
1/1 [==============================] - 0s 37ms/step - loss: 0.6655 - accuracy: 0.6875 - val_loss: 0.6658 - val_accuracy: 0.6562
Epoch 5/50
1/1 [==============================] - 0s 38ms/step - loss: 0.6795 - accuracy: 0.6875 - val_loss: 0.6624 - val_accuracy: 0.6562
Epoch 6/50
1/1 [==============================] - 0s 41ms/step - loss: 0.6573 - accuracy: 0.6875 - val_loss: 0.6555 - val_accuracy: 0.6562
Epoch 7/50
1/1 [==============================] - 0s 40ms/step - loss: 0.6530 - accuracy: 0.6875 - val_loss: 0.6506 - val_accuracy: 0.6562
Epoch 8/50
1/1 [==========

In [28]:
results_train = model3.evaluate(train_img, train_y)

1/1 [==============================] - 0s 21ms/step - loss: 0.5395 - accuracy: 0.6875


In [29]:
results_test = model3.evaluate(test_img, test_y)

1/1 [==============================] - 0s 19ms/step - loss: 0.5457 - accuracy: 0.7188


In [30]:
results_train

[0.5395029783248901, 0.6875]

In [31]:
results_test

[0.5456678867340088, 0.71875]

## Model 4: CNN

In [32]:
model4 = models.Sequential()
model4.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(32, (4, 4), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Flatten())
model4.add(layers.Dense(64, activation='relu'))
model4.add(layers.Dense(1, activation='sigmoid'))

model4.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [34]:
history4 = model4.fit(train_images,
                    train_y,
                    epochs=30,
                    batch_size=32,
                    validation_data=(val_images, val_y))

Epoch 1/30
1/1 [==============================] - 1s 640ms/step - loss: 0.6718 - acc: 0.6875 - val_loss: 0.6631 - val_acc: 0.6562
Epoch 2/30
1/1 [==============================] - 0s 126ms/step - loss: 0.6526 - acc: 0.6875 - val_loss: 0.6551 - val_acc: 0.6562
Epoch 3/30
1/1 [==============================] - 0s 133ms/step - loss: 0.6419 - acc: 0.6875 - val_loss: 0.6502 - val_acc: 0.6562
Epoch 4/30
1/1 [==============================] - 0s 125ms/step - loss: 0.6346 - acc: 0.6875 - val_loss: 0.6470 - val_acc: 0.6562
Epoch 5/30
1/1 [==============================] - 0s 126ms/step - loss: 0.6298 - acc: 0.6875 - val_loss: 0.6449 - val_acc: 0.6562
Epoch 6/30
1/1 [==============================] - 0s 127ms/step - loss: 0.6266 - acc: 0.6875 - val_loss: 0.6436 - val_acc: 0.6562
Epoch 7/30
1/1 [==============================] - 0s 124ms/step - loss: 0.6243 - acc: 0.6875 - val_loss: 0.6428 - val_acc: 0.6562
Epoch 8/30
1/1 [==============================] - 0s 121ms/step - loss: 0.6229 - acc: 0.68

In [ ]:
results_train = model.evaluate(train_images, train_y)

In [ ]:
results_test = model.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

## Overampling